In [1]:
library(Seurat)
library(clustree)
library(dittoSeq)
library(dplyr)

Attaching SeuratObject

Loading required package: ggraph

Loading required package: ggplot2


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
maindir="/Users/rejsmont/Google Drive/My Drive/Projects/RDN-WDP/sharing/"
colors = dittoColors()
options(warn=-1)

In [3]:
counts = read.delim(file.path(maindir, "export", "aerts_counts.csv"), sep=",", header=TRUE, row.names=1, check.names=FALSE)
counts = t(counts)
obj = CreateSeuratObject(counts=counts, project="Eye Disc", min.cells=1, min.features=1)
cells = read.delim(file.path(maindir, "export", "aerts_cells.csv"), sep=",", header=TRUE, row.names=1, check.names=FALSE)
obj@meta.data$barcodes = cells[,1]
obj@meta.data$rep = unlist(lapply(strsplit(cells[,1],"-"), function(x)x[2]))
clusters = read.delim(file.path(maindir, "export", "aerts_clusters.csv"), sep=",", header=TRUE, row.names=1, check.names=FALSE)
obj@meta.data$image_clusters = clusters[,1]

In [4]:
sct <- SCTransform(obj, do.scale=TRUE, return.only.var=FALSE)
write.table(as.matrix(GetAssayData(object = sct, assay.type = "SCT", slot = "scale.data")), 
            'seurat_scale.csv', sep = ',', row.names = T, col.names = T, quote = F)
write.table(as.matrix(GetAssayData(object = sct, assay.type = "SCT", slot = "counts")), 
            'seurat_count.csv', sep = ',', row.names = T, col.names = T, quote = F)
write.table(as.matrix(GetAssayData(object = sct, assay.type = "SCT", slot = "data")), 
            'seurat_norm.csv', sep = ',', row.names = T, col.names = T, quote = F)

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 10329 by 2382

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 2382 cells



  |======================================================================| 100%


Found 129 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 10329 genes



  |======================================================================| 100%


Computing corrected count matrix for 10329 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 23.09356 secs

Determine variable features

Place corrected count matrix in counts slot

Centering and scaling data matrix

Set default assay to SCT



In [5]:
sct_var <- SCTransform(obj, do.scale=TRUE, return.only.var=TRUE)
write.table(as.matrix(GetAssayData(object = sct_var, assay.type = "SCT", slot = "scale.data")), 
            'seurat_var.csv', sep = ',', row.names = T, col.names = T, quote = F)

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 10329 by 2382

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 2382 cells



  |======================================================================| 100%


Found 129 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 10329 genes



  |======================================================================| 100%


Computing corrected count matrix for 10329 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 19.86681 secs

Determine variable features

Place corrected count matrix in counts slot

Centering and scaling data matrix

Set default assay to SCT



In [6]:
res = c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1)
sct_var <- RunPCA(sct_var, verbose = FALSE)
sct_var <- FindNeighbors(sct_var, dims = 1:30, verbose = FALSE)
sct_var <- RunUMAP(sct_var, dims = 1:30, verbose = FALSE, n.components = 3L)

In [7]:
write.table(Embeddings(sct_var, reduction = "pca"), 
            'seurat_pca.csv', sep = ',', row.names = T, col.names = T, quote = F)
write.table(Embeddings(sct_var, reduction = "umap"), 
            'seurat_umap.csv', sep = ',', row.names = T, col.names = T, quote = F)

In [8]:
for(i in 1:length(res)) {
  sct_var <- FindClusters(sct_var, verbose = FALSE, resolution = res[i])
}

write.table(sct_var@meta.data, 'seurat_metadata.csv', sep = ',', row.names = T, col.names = T, quote = F)

In [9]:
for(i in 1:length(res)) {
    Idents(sct_var) = paste("SCT_snn_res.", res[i], sep = "")
    res_seurat = FindAllMarkers(sct_var, logfc.threshold = 0.25, min.pct = 0.25)
    write.table(res_seurat, file=paste("seurat_diff_SCT_snn_res.", res[i], ".csv", sep = ""),
                sep = ',', row.names = T, col.names = T, quote = F)
}

Calculating cluster 0

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calcul

In [11]:
Idents(sct_var) = "image_clusters"
res_seurat = FindAllMarkers(sct_var, logfc.threshold = 0.25, min.pct = 0.25)
write.table(res_seurat, file="seurat_diff_image_clusters.csv",
            sep = ',', row.names = T, col.names = T, quote = F)

Calculating cluster 6

Calculating cluster 7

Calculating cluster 4

Calculating cluster 3

Calculating cluster 5

Calculating cluster 2

Calculating cluster 1

